In [2]:
using JuMP, Cbc

# sets
gases=1:3
crudes=1:3

#data
gasprice=[70,60,50]
minoctane=[10,8,6]
maxsulfer=[1,2,1]
mingas=[3000,2000,1000]
# added the $4 processing cost to crude cost
crudecost=[49,39,29]
crudeoct=[12,6,8]
crudesulf=[0.5,2,3]
maxcrude=[5000,5000,5000]
maxtotal=14000
adbpd=10  # advertising barrels per dollar

# model
m = Model()

@variable(m, blend[crudes,gases] >= 0)
@variable(m, 0 <= crude_buy[i in crudes] <= maxcrude[i])
@variable(m, gas_sell[gases] >= 0)
@variable(m, adspend[gases] >= 0)

@objective(m, Max,
    sum(gasprice[j]*gas_sell[j] for j in gases)
    - sum(crudecost[i]*crude_buy[i] for i in crudes)
    - sum(adspend[j] for j in gases))

@constraint(m, gastot[j in gases],
    sum(blend[i,j] for i in crudes) == gas_sell[j])
@constraint(m, crudetot[i in crudes],
    sum(blend[i,j] for j in gases) == crude_buy[i])
@constraint(m, maxproc,
    sum(crude_buy[i] for i in crudes) <= maxtotal)
@constraint(m, meetdemand[j in gases],
    gas_sell[j] == mingas[j]+adbpd*adspend[j])
@constraint(m, octanelb[j in gases],
    sum(crudeoct[i]*blend[i,j] for i in crudes)
        >= minoctane[j]*gas_sell[j])
@constraint(m, sulfurub[j in gases],
    sum(crudesulf[i]*blend[i,j] for i in crudes)
        <= maxsulfer[j]*gas_sell[j])

set_optimizer(m,Cbc.Optimizer)
optimize!(m)

# print total cost
println("Maximum profit: \$", objective_value(m))

println("production plan: ")
for j in gases
    println("Total of Gas ", j, " to sell: ", round(value(gas_sell[j]),digits=1))
    println("\t Mixed from:")
    for i in crudes
        println("\t \t Crude ", i, ": ", round(value(blend[i,j]),digits=1))
    end
    println("\t Advertising for gas ", j, ": ", round(value(adspend[j]),digits=1))
end

Maximum profit: $287750.0
production plan: 
Total of Gas 1 to sell: 3000.0
	 Mixed from:
	 	 Crude 1: 2000.0
	 	 Crude 2: 1000.0
	 	 Crude 3: 0.0
	 Advertising for gas 1: 0.0
Total of Gas 2 to sell: 9500.0
	 Mixed from:
	 	 Crude 1: 2333.3
	 	 Crude 2: 3666.7
	 	 Crude 3: 3500.0
	 Advertising for gas 2: 750.0
Total of Gas 3 to sell: 1000.0
	 Mixed from:
	 	 Crude 1: 666.7
	 	 Crude 2: 333.3
	 	 Crude 3: 0.0
	 Advertising for gas 3: 0.0
Presolve 13 (-3) rows, 15 (-3) columns and 51 (-6) elements
0  Obj 380000 Primal inf 10333.333 (6) Dual inf 179.7 (3)
12  Obj 287750
Optimal - objective value 287750
After Postsolve, objective 287750, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 287750 - 12 iterations time 0.002, Presolve 0.00
